In [1]:
from sys import path
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import math

if not ("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\" in path) :
    path.append("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\")

from FeedingHabitat.feedingHabitat import FeedingHabitat
%load_ext memory_profiler

In [2]:
fh = FeedingHabitat()
fh.cohorts_number = 1

In [3]:
min, max = 0.0,10.0
fh.variables_dictionary["oxygen_L1"] = np.ndarray((1,2,2), buffer=np.arange(min, max))
fh.variables_dictionary["oxygen_L2"] = np.ndarray((1,2,2), buffer=np.arange(min+1, max+1))
fh.variables_dictionary["oxygen_L3"] = np.ndarray((1,2,2), buffer=np.arange(min+2, max+2))

fh.variables_dictionary["temperature_L1"] = fh.variables_dictionary["oxygen_L1"] 
fh.variables_dictionary["temperature_L2"] = fh.variables_dictionary["oxygen_L2"]
fh.variables_dictionary["temperature_L3"] = fh.variables_dictionary["oxygen_L3"]

In [4]:
print(fh.variables_dictionary["oxygen_L1"])
print(fh.variables_dictionary["oxygen_L2"])
print(fh.variables_dictionary["oxygen_L3"])

[[[0. 1.]
  [2. 3.]]]
[[[1. 2.]
  [3. 4.]]]
[[[2. 3.]
  [4. 5.]]]


In [5]:
fh.setMask()

In [6]:
results = fh.oxygen(0.0001, 3.)
results

array([[[[[1.0000000e-12, 9.9999999e-09],
          [9.9990001e-05, 5.0000000e-01]]],


        [[[9.9999999e-09, 9.9990001e-05],
          [5.0000000e-01, 9.9990001e-01]]],


        [[[9.9990001e-05, 5.0000000e-01],
          [9.9990001e-01, 9.9999999e-01]]]]])

In [7]:
results.shape

(1, 3, 1, 2, 2)

## **Real DATA**

In [7]:
xr_ds_o2 = xr.open_dataset("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\SP\\MFCat_sp_2x30d_interim_historic_O2_L2_1979_2010.nc")

fh.variables_dictionary["oxygen_L1"] = xr_ds_o2['o2'].data[:12,:,:]
fh.variables_dictionary["oxygen_L2"] = xr_ds_o2['o2'].data[:12,:,:]
fh.variables_dictionary["oxygen_L3"] = xr_ds_o2['o2'].data[:12,:,:]

fh.variables_dictionary["temperature_L1"] = xr_ds_o2['o2'].data
fh.variables_dictionary["temperature_L2"] = xr_ds_o2['o2'].data
fh.variables_dictionary["temperature_L3"] = xr_ds_o2['o2'].data

fh.setMask()
fh.partial_oxygen_time_axis = True

In [21]:
%mprun -f fh.oxygen fh.oxygen(0.0001, 3.)

Filename: C:\Users\julesl\Documents (Local)\JupyterLabWorkspace\Library\FeedingHabitat\feedingHabitat.py

Line #    Mem usage    Increment  Occurences   Line Contents
   475    195.3 MiB    195.3 MiB           1       def oxygen(self, gamma, o_star) :
   476                                         
   477    195.3 MiB      0.0 MiB           1           layers = ["oxygen_L1", "oxygen_L2", "oxygen_L3"]
   478                                                 
   479    195.3 MiB      0.0 MiB           1           cohort_buffer = []
   480                                                 
   481    195.3 MiB      0.0 MiB           1           if self.cohorts_to_compute is None :
   482    195.3 MiB      0.0 MiB           1               cohorts_number = self.cohorts_number
   483                                                 else :
   484                                                     cohorts_number = len(self.cohorts_to_compute)
   485    195.5 MiB      0.0 MiB           2           

In [8]:
results = fh.oxygen(0.0001, 3.)

In [9]:
results.shape

(1, 3, 12, 32, 80)

In [10]:
buffer = []
for time_unit in range(fh.variables_dictionary["temperature_L1"].shape[0]) :
    buffer.append(results[:,:,time_unit % results.shape[2],:,:])
ha_result = np.transpose(np.array(buffer),(1,2,0,3,4))

In [11]:
ha_result.shape

(1, 3, 384, 32, 80)

In [13]:
results[:,:,:12,:,:].shape

(1, 3, 12, 32, 80)

In [14]:
ha_result[:,:,:12,:,:].shape

(1, 3, 12, 32, 80)

In [21]:
False in (results[:,:,:,:,:] == ha_result[:,:,12:24,:,:])

False

## **Many cohorts**

In [5]:
fh.cohorts_number = 37

In [6]:
%mprun -f fh.oxygen fh.oxygen(0.0001, 3.)

Filename: C:\Users\julesl\Documents (Local)\JupyterLabWorkspace\Library\FeedingHabitat\feedingHabitat.py

Line #    Mem usage    Increment  Occurences   Line Contents
   243    141.4 MiB    141.4 MiB           1       def oxygen(self, gamma, o_star) :
   244                                         
   245    141.4 MiB      0.0 MiB           1           layers = ["oxygen_L1", "oxygen_L2", "oxygen_L3"]
   246                                                 
   247    141.4 MiB      0.0 MiB           1           cohort_buffer = []
   248    558.1 MiB      0.0 MiB          38           for cohort in range(self.cohorts_number) :
   249    546.8 MiB      0.0 MiB          37               layer_buffer = []
   250    558.1 MiB      0.0 MiB         148               for layer, mask in zip(layers, self.global_mask.keys()) :
   251    554.4 MiB      0.0 MiB         111                   variable = self.variables_dictionary[layer]
   252                                                         #Tak

In [13]:
results.shape

(37, 3, 384, 32, 80)

In [14]:
results.size

109117440